##**4 Assignment**
a) 아래 예제 코드를 이용해 텍스트 입력의 숫자 변환 과정을 체크한다

b) testset의 임의 입력을 LongTensor로 변환해 학습 완료된 모델에 입력해보고, 결과가 어떠한지 체크한다 (하단의 출력 결과를 참조)

In [1]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data, datasets 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
batch_size = 64
learning_rate = 0.001
training_epochs = 10

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(trainset, min_freq=5)
LABEL.build_vocab(trainset)

trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=batch_size,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 2 

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers 

        self.embed = nn.Embedding(n_vocab, embed_dim)

        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        
        x = self.embed(x)
        x, _ = self.gru(x)  

        h_t = x[:,-1,:]

        self.dropout(h_t)

        out = self.out(h_t) 
        return out

model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)

criterion = nn.CrossEntropyLoss().to(device)    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  avg_cost = 0
  for batch in train_iter:
    X, Y = batch.text.to(device), batch.label.to(device)
    Y.data.sub_(1)
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / batch_size
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

torch.save(model.state_dict(), '/content/drive/MyDrive/인공지능응용/hw14_model_s1.pt')

model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/인공지능응용/hw14_model_s1.pt'))

corrects = 0
for batch in val_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 

print('accuracy = ', corrects/len(val_iter.dataset)*100.0)

corrects = 0
for batch in test_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 

print('accuracy = ', corrects/len(val_iter.dataset)*100.0)

input_text = testset[3].text
print(input_text)
for i in range(len(input_text)):
  print(TEXT.vocab[input_text[i]], end = ', ')

result = []
for i in range(len(input_text)):
  result.append(TEXT.vocab[input_text[i]])
print(" ", end='\n')
print('----------------------')

result = torch.LongTensor(result)
result = result.unsqueeze(0).to(device)

hypothesis = model_new(result)
print(hypothesis.max(1)[1])

cuda
downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:02<00:00, 39.5MB/s]


[TrainSet]: 20000 [ValSet]: 5000 [TestSet]: 25000 [Vocab]: 46159 [Classes] 2
[Epoch:    1] cost = 3.41717672
[Epoch:    2] cost = 3.39704394
[Epoch:    3] cost = 3.38808107
[Epoch:    4] cost = 3.39739299
[Epoch:    5] cost = 3.39174867
[Epoch:    6] cost = 3.39224935
[Epoch:    7] cost = 2.93911386
[Epoch:    8] cost = 1.53755748
[Epoch:    9] cost = 0.798684776
[Epoch:   10] cost = 0.374675035
accuracy =  tensor(86.9600, device='cuda:0')
accuracy =  tensor(431.2000, device='cuda:0')
['i', 'am', 'a', '20', 'year', 'old', 'bloke', 'from', 'england.', 'i', "don't", 'cry', 'at', 'films.', 'but', 'this', 'one', 'moved', 'me.', 'i', 'cried', 'like', 'a', 'girl!', 'this', 'is', 'absolutely', 'the', 'most', 'moving', 'film', 'i', 'have', 'ever', 'seen,', 'even', 'though', 'the', 'story', 'was', 'questionable.', 'joseph', "mazello's", 'little', 'face', 'when', 'he', 'dreams', 'of', 'crying', 'made', 'me', 'sob', 'every', 'time.', 'how', 'could', 'anyone', 'hurt', 'such', 'a', 'sweet', 'lookin